In [1]:
!pip install llama-index==0.8.35 llama-hub pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of llama-hub to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-hub to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 843.5/843.5 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━

In [ ]:
# setup OpenAI
import openai

openai.api_key = "sk-"

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

## Multi-Document Comparisons


In [ ]:
from llama_index import SimpleDirectoryReader, ServiceContext, VectorStoreIndex
from llama_index.llms import OpenAI

from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.query_engine import SubQuestionQueryEngine

In [ ]:
llm = OpenAI(temperature=0, model="gpt-3.5-turbo", max_tokens=-1)
service_context = ServiceContext.from_defaults(llm=llm)

In [ ]:
!wget https://www.dropbox.com/s/qctkz6nxhm0y5qe/lyft_2021.pdf?dl=1 -O lyft_2021.pdf
!wget https://www.dropbox.com/s/te0a2w227v27iag/uber_2021.pdf?dl=1 -O uber_2021.pdf

--2023-09-27 16:12:22--  https://www.dropbox.com/s/qctkz6nxhm0y5qe/lyft_2021.pdf?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/qctkz6nxhm0y5qe/lyft_2021.pdf [following]
--2023-09-27 16:12:22--  https://www.dropbox.com/s/dl/qctkz6nxhm0y5qe/lyft_2021.pdf
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4d71ed4051034c17980b7016f8.dl.dropboxusercontent.com/cd/0/get/CEiuUsHMkMm2woAYiLVSkQFOYF0PW2ZBZ2jOY9CBTnYdESsrjJnG9i3iZNEBeH_dkLjlQi1mA8YFgekgP6rjlg1i-rUiNb_3bKhmXoaNHkzngpRbRW6iGa0081n2TcOCsYxLh40EEWs4z_jvqfDBZpJC/file?dl=1# [following]
--2023-09-27 16:12:23--  https://uc4d71ed4051034c17980b7016f8.dl.dropboxusercontent.com/cd/0/get/CEiuUsHMkMm2woAYiLVSkQFOYF0PW2ZBZ2jOY9CBTnYdESsrjJnG9i3iZNEBeH_dkLjlQi1mA8YFgekgP6rjlg1i-rUiNb_3b

In [ ]:
lyft_docs = SimpleDirectoryReader(input_files=["lyft_2021.pdf"]).load_data()
uber_docs = SimpleDirectoryReader(input_files=["uber_2021.pdf"]).load_data()

In [ ]:
print(lyft_docs[0].get_content())

UNITED STATESSECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K
 
(Mark One)
☒
ANNUAL REPORT PURS UANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934For the fiscal year ended December 31, 2021
OR
☐
TRANSITION REPORT PURS UANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 FOR THE TRANSITION PERIODFROM                      TO
Commission File Number 001-38846
Lyft, Inc.
(Exact name of Registrant as specified in i
ts Charter)Delaware
20-8809830 (State or other jurisdiction of
incorporation or organization)
(I.R.S. EmployerIdentification No.)
185 Berry Street, Suite 5000
San Francisco, California
94107 (Address of principal executive offices)
(Zip Code) Registrant’s telephone number, including area code: (844) 250
-2773Securities registered pursuant to Section 12(b) of the Act: 
Title of each class
TradingSymbol(s)
Name of each exchange on which registered Class A common stock, par value of $0.00001 per share
LYFT Nasdaq Global Select Market S

### Setup Baseline Query Engine

We first setup a top-k query engine.

In [ ]:
# setup baseline index
base_index = VectorStoreIndex.from_documents(lyft_docs + uber_docs)
base_engine = base_index.as_query_engine(similarity_top_k=4)

In [ ]:
response = base_engine.query("What are some risk factors for Uber?")
print(str(response))

Some risk factors for Uber include criminal, violent, inappropriate, or dangerous activity by platform users, which could harm their reputation and business. They are unable to control or predict the actions of platform users and third parties, which may result in injuries, property damage, or loss of life. The qualification processes and background checks for drivers may not expose all relevant information, and there have been allegations of inadequate qualification processes and background checks. Engaging in criminal activity or misconduct by drivers or consumers may also impact their brand and reputation. Additionally, public reporting or disclosure of safety incidents on their platform may adversely affect their business. Changes in driver qualification and background-check requirements may increase costs and reduce their ability to onboard new drivers. Maintaining and enhancing their brand and reputation is critical to their business prospects. They have previously received negat

In [ ]:
print(len(response.source_nodes))

4


In [ ]:
print(response.source_nodes[0].get_content())

If platform users engage in, or are subject to, criminal, violent, inappropriate, or dangerous activity that results in major safety incidents, our ability to attractand
 retain Drivers, consumers, merchants, shippers, and carriers may be harmed, which could have an adverse impact on our reputation, business, financialcondition, and operating results.
We
 are not able to control or predict the actions of platform users and third parties, either during their use of our platform or otherwise, and we may be unableto protect
 or provide a safe environment for Drivers and consumers as a result of certain actions by Drivers, consumers, merchants, carriers, and third parties. Suchactions
 may  result  in  injuries,  property  damage,  or  loss  of  life  for  consumers  and  third  parties,  or  business  interruption,  brand  and  reputational  damage,  orsignificant
 liabilities for us. Although we administer certain qualification processes for users of our platform, including background ch

In [ ]:
response = base_engine.query("Compare and contrast the risk factors of Uber and Lyft?")
print(str(response))

I'm sorry, but I cannot provide a direct comparison and contrast of the risk factors of Uber and Lyft based on the given context information.


In [ ]:
print(response.source_nodes[3].get_content(metadata_mode="all"))

page_label: 19
file_name: lyft_2021.pdf

Risks Related to Operational FactorsOur limited operating history and our evolving business make 
it difficult to evaluate our future prospects and the risks and challenges we may encounter.While
 we have primarily focused on ridesharing since our ridesharing marketplace launched in 2012, our business continues to evolve. We regularly expand ourplatform
 features, offerings and services and change our pricing methodologies. In recent periods, we have also reevaluated and changed our cost structure and focused ourbusiness
 model. Our evolving business, industry and markets make it difficult to evaluate our future prospects and the risks and challenges we may encounter. Risks andchallenges we have faced o
r expect to face include our ability to:•
forecast our revenue and budget for and manage our expenses;•
attract new qualified drivers and new riders and retain existing qualified drivers and existing riders in a cost-effective manner;•
comply wit

In [ ]:
print(response.source_nodes[1].get_content(metadata_mode="all"))

page_label: 10
file_name: lyft_2021.pdf

entrants in the market that may be well-established and enjoy greater resources or other strategic advantages. If we are unable to anticipate or successfully react to thesecompetitive
 challenges in a timely manner, our competitive position could weaken, or fail to improve, and we could experience a decline in revenue or growth stagnationthat could adversely af
fect our business, financial condition and results of operations.Our
 main ridesharing competitors in the United States and Canada include Uber and Via. Our main competitors in the bike and scooter sharing market includeLime
 and Bird. Our main competitors in the consumer vehicle rental market include Enterprise, Hertz and Avis Budget Group as well as emerging car-share marketplaces.We also compete with ce
rtain non-ridesharing transportation network companies and taxi cab and livery companies as well as traditional automotive manufacturers.Additionally,
 there  are  other  non-U.S.-based

### Setup SubQuestionQueryEngine

This query engine does the following:
- Can treat different documents as different "tools"
- Breaks down a complex question into sub-questions over any subset of different documents
- Does retrieval independently per document
- Combines results at the end.

In [ ]:
lyft_index = VectorStoreIndex.from_documents(lyft_docs)

In [ ]:
uber_index = VectorStoreIndex.from_documents(uber_docs)

In [ ]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=2)

In [ ]:
uber_engine = uber_index.as_query_engine(similarity_top_k=2)

In [ ]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description="Provides information about Lyft financials for year 2021",
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description="Provides information about Uber financials for year 2021",
        ),
    ),
]

s_engine = SubQuestionQueryEngine.from_defaults(query_engine_tools=query_engine_tools)

### Try some Queries

We compare our SubQuestionQueryEngine against the base top-k vector query engine.

In [ ]:
response = s_engine.query("Compare and contrast the risk factors of Uber and Lyft?")
print(str(response))

Generated 2 sub questions.
[uber_10k] Q: What are the risk factors for Uber?
[lyft_10k] Q: What are the risk factors for Lyft?
[lyft_10k] A: Lyft faces various risk factors that could potentially harm its business operations and financial condition. These risk factors include general economic factors such as the impact of the COVID-19 pandemic, natural disasters, economic downturns, and political crises. Operational factors also pose risks, including Lyft's limited operating history, financial performance, competition in the industry, unpredictability of results, uncertainty regarding market growth, ability to attract and retain qualified drivers and riders, insurance coverage, autonomous vehicle technology, reputation and brand, illegal or improper activity on the platform, accuracy of background checks, changes to pricing practices, management of growth, security and privacy breaches, reliance on third parties, and the development and management of new offerings on the platform. Thes

While the base query engine couldn't generate a proper response, here we see that the response is much more detailed.

Here's some other comparison queries.

In [ ]:
query_str = "Tell me what was higher, Uber's revenue growth or Lyft's revenue growth, and using the text explain the reasons for revenue growth."
base_response = base_engine.query(query_str)
response = s_engine.query(query_str)


Generated 4 sub questions.
[uber_10k] Q: What was Uber's revenue growth for year 2021?
[lyft_10k] Q: What was Lyft's revenue growth for year 2021?
[uber_10k] Q: What are the factors that contributed to Uber's revenue growth?
[lyft_10k] Q: What are the factors that contributed to Lyft's revenue growth?
[uber_10k] A: Uber's revenue growth for the year 2021 was 57%.
[lyft_10k] A: Lyft's revenue growth for the year 2021 was 36%.
[lyft_10k] A: Lyft's revenue growth can be attributed to several factors. These factors include the popularity, utility, ease of use, performance, and reliability of Lyft's offerings compared to those of its competitors. Additionally, Lyft's reputation, including the perceived safety of its platform, and brand strength relative to its competitors, play a significant role in driving revenue growth. The pricing models and fees charged to drivers on the Lyft platform also contribute to revenue growth. Furthermore, Lyft's ability to attract and retain qualified drivers

In [ ]:
print(f"Base query engine: {str(base_response)}\n\n")
print(f"Sub-question query engine: {str(response)}\n\n")

Base query engine: Uber's revenue growth was higher than Lyft's revenue growth. The text explains that Uber's revenue growth was driven by factors such as growing supply and demand on their platform, increasing existing platform users' activity, introducing their platform to new markets, providing high-quality support to drivers and consumers, expanding their business and increasing market share, competing with competitors' products and offerings, developing new products and technologies, acquiring or investing in businesses that complement their platform, penetrating suburban and rural areas, reducing costs of their Mobility offering, maintaining local regulations, and expanding operations in key countries. On the other hand, Lyft's revenue growth depends on factors such as the popularity and utility of their offerings compared to competitors, their reputation and brand strength, pricing models and fees charged to drivers, managing business operations during the COVID-19 pandemic, att

In [ ]:
print(base_response.source_nodes[3].get_content(metadata_mode="all"))

page_label: 98
file_name: uber_2021.pdf

Note 2 – RevenueThe
 following  tables  present  our  revenues  disaggregated  by  offering  and  geographical  region.  Revenue  by  geographical  region  is  based  on  where  thetransaction
 occurred. This level of disaggregation takes into consideration how the nature, amount, timing, and uncertainty of revenue and cash flows are affectedby economic factors. Revenue 
is presented in the following tables for the years ended December 31, 2019, 2020 and 2021, respectively (in millions):Year Ended December 31,
2019
2020 2021 Mobility revenue 
$ 10,707 $ 6,089 $ 6,953 Delivery revenue 
1,401 3,904 8,362 Freight revenue
731 1,011 2,132 All Other revenue
161 135 8 Total revenue
$ 13,000 $ 11,139 $ 17,455  We
 offer subscription memberships to end-users including Uber One, Uber Pass, Rides Pass, and Eats Pass (“Subscription”). We recognize Subscriptionfees
 ratably over the life of the pass. We allocate Subscription fees earned to Mobility and Deliv

## Complex Document (with Embedded Tables)

In this setting we walk through a document that has an embedded table inside of it.

In [ ]:
!apt install ghostscript python3-tk

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-tk is already the newest version (3.10.8-1~22.04).
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 libgs9 libgs9-common libidn12 libijs-0.35
  libjbig2dec0 poppler-data
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre ghostscript-x poppler-utils
  fonts-japanese-mincho | fonts-ipafont-mincho fonts-japanese-gothic | fonts-ipafont-gothic
  fonts-arphic-ukai fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript libgs9 libgs9-common libidn12
  libijs-0.35 libjbig2dec0 poppler-data
0 upgraded, 10 newly installed, 0 to remove and 19 not upgraded.
Need to get 16.7 MB of archives.
After this operation, 63.0 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 

In [ ]:
!pip install camelot-py pymupdf frontend ghostscript

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 129.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.4/277.4 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.


In [ ]:
import camelot
from llama_index import Document, SummaryIndex

# https://en.wikipedia.org/wiki/The_World%27s_Billionaires
from llama_index import VectorStoreIndex, ServiceContext, LLMPredictor
from llama_index.query_engine import PandasQueryEngine, RetrieverQueryEngine
from llama_index.retrievers import RecursiveRetriever
from llama_index.schema import IndexNode
from llama_index.llms import OpenAI

from llama_hub.file.pymu_pdf.base import PyMuPDFReader
from pathlib import Path
from typing import List

ModuleNotFoundError: ignored

### Parse out Table, build Pandas Query Engine

In [ ]:
!wget "https://www.dropbox.com/scl/fi/waoz9bo9yiemnhnqvu0cc/billionaires_page.pdf?rlkey=4i08msa7zr1lpnuq2y1vs2xgw&dl=1" -O billionaires_page.pdf

--2023-09-27 07:27:15--  https://www.dropbox.com/scl/fi/waoz9bo9yiemnhnqvu0cc/billionaires_page.pdf?rlkey=4i08msa7zr1lpnuq2y1vs2xgw&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/e/scl/fi/waoz9bo9yiemnhnqvu0cc/billionaires_page.pdf?rlkey=4i08msa7zr1lpnuq2y1vs2xgw&dl=1 [following]
--2023-09-27 07:27:15--  https://www.dropbox.com/e/scl/fi/waoz9bo9yiemnhnqvu0cc/billionaires_page.pdf?rlkey=4i08msa7zr1lpnuq2y1vs2xgw&dl=1
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6ebcac97d77b6bb40197e1e0b8.dl.dropboxusercontent.com/cd/0/inline/CEjd0bmHblyQItvTkgw86sgszJJ_9AvEwqJuPP43UVonHMvqVyL2fKuYOKCqxLyt74fbcmpgA5aPkvLMHbavZHkdylda6KYLnGbOk-tK0phTVcabs55yvI-yHYUVemRzi8A/file?dl=1# [following]
--2023-09-27 07:27:16--  htt

In [ ]:
# initialize PDF reader
reader = PyMuPDFReader()

In [ ]:
file_path = "billionaires_page.pdf"
docs = reader.load(file_path=file_path)

In [ ]:
# use camelot to parse tables
def get_tables(path: str, pages: List[int]):
    table_dfs = []
    for page in pages:
        table_list = camelot.read_pdf(path, pages=str(page))
        table_df = table_list[0].df
        table_df = (
            table_df.rename(columns=table_df.iloc[0])
            .drop(table_df.index[0])
            .reset_index(drop=True)
        )
        table_dfs.append(table_df)
    return table_dfs

In [ ]:
table_dfs = get_tables(file_path, pages=[3, 25])

In [ ]:
# shows list of top billionaires in 2023
table_dfs[0]

,No.,Name,Net worth\n(USD),Age,Nationality,Primary source(s) of wealth
0,1,Bernard Arnault &\nfamily,$211 billion,74,France,LVMH
1,2,Elon Musk,$180 billion,51,United\nStates,"Tesla, SpaceX, X Corp."
2,3,Jeff Bezos,$114 billion,59,United\nStates,Amazon
3,4,Larry Ellison,$107 billion,78,United\nStates,Oracle Corporation
4,5,Warren Buffett,$106 billion,92,United\nStates,Berkshire Hathaway
5,6,Bill Gates,$104 billion,67,United\nStates,Microsoft
6,7,Michael Bloomberg,$94.5 billion,81,United\nStates,Bloomberg L.P.
7,8,Carlos Slim & family,$93 billion,83,Mexico,"Telmex, América Móvil, Grupo\nCarso"
8,9,Mukesh Ambani,$83.4 billion,65,India,Reliance Industries
9,10,Steve Ballmer,$80.7 billion,67,United\nStates,Microsoft


In [ ]:
# shows list of top billionaires
table_dfs[1]

,Year,Number of billionaires,Group's combined net worth
0,2023[2],"2,640",$12.2 trillion
1,2022[6],"2,668",$12.7 trillion
2,2021[11],"2,755",$13.1 trillion
3,2020,"2,095",$8.0 trillion
4,2019,"2,153",$8.7 trillion
5,2018,"2,208",$9.1 trillion
6,2017,"2,043",$7.7 trillion
7,2016,"1,810",$6.5 trillion
8,2015[18],"1,826",$7.1 trillion
9,2014[67],"1,645",$6.4 trillion


In [ ]:
# define query engines over these tables
df_query_engines = [PandasQueryEngine(table_df) for table_df in table_dfs]

In [ ]:
response = df_query_engines[0].query(
    "What's the net worth of the second richest billionaire in 2023?"
)
print(str(response))

$180 billion


In [ ]:
response = df_query_engines[1].query("How many billionaires were there in 2009?")
print(str(response))

793


### Build Recursive Retriever

We define a top-level vector index that does top-k lookup over a set of Nodes. We define two special nodes (`IndexNode` objects) linking to each of these tables.

We define a `RecursiveRetriever` object to recursively retrieve/query nodes. We then put this in our `RetrieverQueryEngine` along with a `ResponseSynthesizer` to synthesize a response.

We pass in mappings from id to retriever and id to query engine. We then pass in a root id representing the retriever we query first.

In [ ]:
llm = OpenAI(temperature=0, model="gpt-4")

service_context = ServiceContext.from_defaults(
    llm=llm,
)

In [ ]:
doc_nodes = service_context.node_parser.get_nodes_from_documents(docs)

In [ ]:
# define index nodes
summaries = [
    "This node provides information about the world's richest billionaires in 2023",
    "This node provides information on the number of billionaires and their combined net worth from 2000 to 2023.",
]

df_nodes = [
    IndexNode(text=summary, index_id=f"pandas{idx}")
    for idx, summary in enumerate(summaries)
]

df_id_query_engine_mapping = {
    f"pandas{idx}": df_query_engine
    for idx, df_query_engine in enumerate(df_query_engines)
}

In [ ]:
# construct top-level vector index + query engine
vector_index = VectorStoreIndex(doc_nodes + df_nodes)
vector_retriever = vector_index.as_retriever(similarity_top_k=1)

In [ ]:
from llama_index.retrievers import RecursiveRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.response_synthesizers import get_response_synthesizer

recursive_retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever},
    query_engine_dict=df_id_query_engine_mapping,
    verbose=True,
)

response_synthesizer = get_response_synthesizer(
    # service_context=service_context,
    response_mode="compact"
)

query_engine = RetrieverQueryEngine.from_args(
    recursive_retriever, response_synthesizer=response_synthesizer
)

### Define Baseline Retriever

We also define a baseline retriever that does top-k lookup over the raw document.

In [ ]:
# baseline vector index (that doesn't include the extra df nodes).
# used to benchmark
vector_index0 = VectorStoreIndex(doc_nodes)
vector_query_engine0 = vector_index0.as_query_engine()

### Compare Results

We compare results between the recursive retriever vs. baseline retriever.

In [ ]:
response = query_engine.query(
    "How many billionaires were there in 2009?"
)
print(str(response))

Retrieving with query id None: How many billionaires were there in 2009?
Retrieved node with id, entering: pandas1
Retrieving with query id pandas1: How many billionaires were there in 2009?
Got response: 793
793


In [ ]:
response.source_nodes[0].node.get_content()

In [ ]:
response = vector_query_engine0.query(
    "How many billionaires were there in 2009?"
)
print(str(response))

Based on the context information, it is not possible to determine the exact number of billionaires in 2009.


In [ ]:
print(response.source_nodes[1].node.get_content())

In [ ]:
response = query_engine.query(
    "What is the average age of top 5 billionaires in 2023? Make sure age is a float."
)
print(str(response))

Retrieving with query id None: What is the average age of top 5 billionaires in 2023? Make sure age is a float.
Retrieved node with id, entering: pandas0
Retrieving with query id pandas0: What is the average age of top 5 billionaires in 2023? Make sure age is a float.
Got response: 70.8
The average age of the top 5 billionaires in 2023 is 70.8.


In [ ]:
response = vector_query_engine0.query(
    "What is the average age of top 5 billionaires in 2023? Make sure age is a float."
)
print(str(response))

The average age of the top 5 billionaires in 2023 is 72.2.


Of course, just like the baseline vector query engine, the recursive retriever can answer semantic queries over the article as well.

In [ ]:
response = query_engine.query(
    "How is wealth accounted for in recipients if the billionaire is deceased?"
)
print(str(response))

Retrieving with query id None: How is wealth accounted for in recipients if the billionaire is deceased?
Retrieving text node: 7/1/23, 11:31 PM
The World's Billionaires - Wikipedia
https://en.wikipedia.org/wiki/The_World%27s_Billionaires
2/33
stock are priced to market on a date roughly a month before publication. Privately held companies are
priced by the prevailing price-to-sales or price-to-earnings ratios. Known debt is subtracted from
assets to get a final estimate of an individual's estimated worth in United States dollars. Since stock
prices fluctuate rapidly, an individual's true wealth and ranking at the time of publication may vary
from their situation when the list was compiled.[7]
When a living individual has dispersed his or her wealth to immediate family members it is included
under a single listing (as a single "family fortune") provided that individual (the grantor) is still living.
However, if a deceased billionaire's fortune has been dispersed, it will not appear as a

In [ ]:
response = vector_query_engine0.query(
    "How is wealth accounted for in recipients if the billionaire is deceased?"
)
print(str(response))

If a deceased billionaire's fortune has been dispersed, it will not appear as a single listing. Instead, each recipient will only appear if their own total net worth is over a billion dollars. In other words, the net worth of each individual recipient will not be combined with that of other family members.


## Multi-modal

In [ ]:
from unstructured.partition.pdf import partition_pdf